# 미세 튜닝 모델 검증

In [ ]:
import torch
import requests
from PIL import Image
from io import BytesIO
from matplotlib import pyplot as plt
import transformers

In [ ]:
# Set device
device = (
    "mps"
    if torch.backends.mps.is_available()
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
torch_dtype = torch.float16 if device == "cuda" else torch.float16

In [ ]:
print(device)

# Text2Image

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, UNet2DConditionModel

model_path = "/Volumes/jmoh2T/models/sd-foodai-model"
unet = UNet2DConditionModel.from_pretrained(model_path + "/checkpoint-25000/unet", torch_dtype=torch_dtype)

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", unet=unet, torch_dtype=torch_dtype)
pipe.to(device)

# image = pipe(prompt="yoda").images[0]
# image.save("yoda-naruto.png")

In [ ]:
# Load the pipeline
model_id_or_path = "CompVis/stable-diffusion-v1-4"
original_pipe = StableDiffusionPipeline.from_pretrained(model_id_or_path, torch_dtype=torch_dtype).to(device)

In [ ]:
# prompt = "Seafood hot pot, as if it's being served in a Korean restaurant"
# prompt = "Pajeon (green onion pancake), Korean style"
# prompt = "Songpyeon (half-moon shaped rice cake), Korean style, photographed from directly above"
# prompt = 'Small clams, steamed, photographed from directly above'
# prompt = "Roasted seaweed, Korean style, photographed from directly above"
# prompt = 'Tteokbokki (spicy rice cakes) with a half of boiled egg on it, which is Korean street food, photographed from directly above'
# prompt = "Ramyeon (Korean instant noodles), photographed from directly above"
# prompt = "Kimchi soup, photographed from directly above, without chopsticks"
# prompt = "Naengmyeon (cold noodles) with a cup of coffe at left side, photographed from directly above"
# prompt = "Korean favoriate food called 'Chicken feet' with hot source, unurgly, photographed from directly above" # 혐오
# 
# prompt = "Spicy pork bulgogi, a food which is photographed from directly above"

In [ ]:
generator = torch.Generator(device=device).manual_seed(42)
pipe(prompt=prompt,
    # negative_prompt="chinese food", # Negative prompt to avoid
    height=480, width=480,     # Specify the image size
    guidance_scale=8,          # How strongly to follow the prompt
    num_inference_steps=35,    # How many steps to take
    generator=generator        # Fixed random seed
    ).images[0]

In [ ]:
generator = torch.Generator(device=device).manual_seed(42)
original_pipe(prompt=prompt, 
    height=480, width=480,     # Specify the image size
    guidance_scale=8,          # How strongly to follow the prompt
    num_inference_steps=35,    # How many steps to take
    generator=generator        # Fixed random seed
    ).images[0]

In [ ]:
#@markdown comparing guidance scales:
cfg_scales = [1.1, 8, 12, 20] #@param
# prompt = "A collie with a pink hat" #@param
fig, axs = plt.subplots(1, len(cfg_scales), figsize=(16, 5))
for i, ax in enumerate(axs):
  im = pipe(prompt, height=480, width=480,
    guidance_scale=cfg_scales[i], num_inference_steps=35,
    generator=torch.Generator(device=device).manual_seed(42)).images[0]
  ax.imshow(im); ax.set_title(f'CFG Scale {cfg_scales[i]}');

# 조사: 한국어 스테이블 디퓨전

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler

repo = "Bingsu/my-korean-stable-diffusion-v1-5"
euler_ancestral_scheduler = EulerAncestralDiscreteScheduler.from_config(repo, subfolder="scheduler")
ko_pipe = StableDiffusionPipeline.from_pretrained(
    repo, scheduler=euler_ancestral_scheduler, torch_dtype=torch.float16,
)
ko_pipe.to(device)

In [ ]:
generator = torch.Generator(device=device).manual_seed(42)
ko_pipe(prompt="고양이 한마리",
    # negative_prompt="chinese food", # Negative prompt to avoid
    height=480, width=480,     # Specify the image size
    guidance_scale=8,          # How strongly to follow the prompt
    num_inference_steps=35,    # How many steps to take
    generator=generator        # Fixed random seed
    ).images[0]